Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [3]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [4]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [5]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [6]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [7]:
# Take a look at the data
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019
23039,4,OTHER,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,NaN,R2,"61-05 39TH AVENUE, F5",F5,11377.0,1.0,0.0,85.0,"15,151",854.0,1927.0,2,R2,569202,04/30/2019


In [8]:
# Import datetime libary
import datetime as dt

# Convert the SALE_DATE to datetime format
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

In [12]:
# Create the train data
train = df[df['SALE_DATE'].dt.month <= 3]

# Create the test data
test = df[df['SALE_DATE'].dt.month == 4]

In [10]:
# Create the target and features for train data
target = 'SALE_PRICE'
# Features for train and test
features1 = train.columns.drop(target)
features2 = test.columns.drop(target)

# Create the matrix for y and X(1:train 2:test)
y1 = train[target]
X1 = train[features1]
y2 = test[target]
X2 = test[features2]

# Import for sklearn the train test split 
from sklearn.model_selection import train_test_split

# Declare the variable for the slipt
X1_train, X1_val, y1_train, y1_val = train_test_split(X1,y1, test_size=0.25, random_state=19)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2,y2, test_size=0.25, random_state=19)

# Print the shape for X_train and y_train
print(X1_train.shape)
print(y1_train.shape)

(13625, 20)
(13625,)


In [ ]:
# Import Encoder from sklearn and category
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

# Instantiate
enc = OneHotEncoder(sparse=False)

# Select and reshape input array
X_train1 = X1
X_test1 = X2

# Use the fit_transform method (2 steps in 1)
encoder = ce.OneHotEncoder(use_cat_names=True)
X_homes_train = encoder.fit_transform(X_train1)
X_homes_test = encoder.fit_transform(X_test1)
X_homes_train.shape, X_homes_test.shape

In [13]:
#  one-hot encoding
# Imports
import numpy as np

# Select and reshape input array
train = train.NEIGHBORHOOD[:, np.newaxis]
test = test.NEIGHBORHOOD[:, np.newaxis]

# Import the encoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

# Instantiate the encoder as an object
enc = OneHotEncoder(sparse=False)
encoder = ce.OneHotEncoder(use_cat_names=True)

# Use the fit_transform method (2 steps in 1)
onehot1 = encoder.fit_transform(train)
onehot2 = encoder.fit_transform(test)

# Display every 25th row
onehot1[::10], onehot2[::10]

(       0_OTHER  0_UPPER EAST SIDE (59-79)  ...  0_FLUSHING-NORTH  0_GRAMERCY
 0            1                          0  ...                 0           0
 10           1                          0  ...                 0           0
 20           1                          0  ...                 0           0
 30           1                          0  ...                 0           0
 40           1                          0  ...                 0           0
 ...        ...                        ...  ...               ...         ...
 18120        1                          0  ...                 0           0
 18130        1                          0  ...                 0           0
 18140        1                          0  ...                 0           0
 18150        1                          0  ...                 0           0
 18160        0                          0  ...                 0           0
 
 [1817 rows x 11 columns],
       0_OTHER  0_GRAMERCY  ...  0_

In [16]:
onehot1.shape

(18167, 11)

In [14]:
# SelectKBest
# Import the feature selector utility
from sklearn.feature_selection import SelectKBest, f_regression

# Create the selector object with the best k=1 features
selector = SelectKBest(score_func=f_regression, k=1)

# Run the selector on the training data
X_train_selected = selector.fit_transform(X1_train, y1_train)

# Find the features that was selected
selected_mask = selector.get_support()
all_features = X1_train.columns
selected_feature = all_features[selected_mask]

print('The selected feature: ', selected_feature[0])

ValueError: ignored